# Product Multimodal Search using Amazon Nova Embedding and OpenSearch Serverless
Demonstrate multimodal product search using text descriptions and product images from Amazon Berkeley Objects dataset.

- Amazon Nova Mutlimodal Embedding Model
- OpenSearch Serverless Vector DB


![Nova MM Embedding](./images/visual-product-search.png)

In [ ]:
!pip install boto3 pandas opensearch-py requests-aws4auth --upgrade

In [ ]:
import boto3
import pandas as pd
import json
import random
import time
from datetime import datetime
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

model_id = 'amazon.nova-2-multimodal-embeddings-v1:0'
region = 'us-east-1'
dim = 3072
index_name = "product-multimodal-index"

bedrock = boto3.client("bedrock-runtime", region_name=region)
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, 'aoss', session_token=credentials.token)

role_arn = boto3.client('sts').get_caller_identity()['Arn']
aoss_client = boto3.client('opensearchserverless', region_name=region)
suffix = random.randrange(200, 900)

## 1. Create OpenSearch Serverless Collection and Index

In [ ]:
def create_policies_in_oss(vector_store_name, aoss_client, role_arn):
    encryption_policy_name = f"nova-mm-sample-sp-{suffix}"
    network_policy_name = f"nova-mm-sample-np-{suffix}"
    access_policy_name = f'nova-mm-sample-ap-{suffix}'

    try:
        encryption_policy = aoss_client.create_security_policy(
            name=encryption_policy_name,
            policy=json.dumps({
                'Rules': [{'Resource': ['collection/' + vector_store_name], 'ResourceType': 'collection'}],
                'AWSOwnedKey': True
            }),
            type='encryption'
        )
    except Exception as ex:
        print(ex)
    
    try:
        network_policy = aoss_client.create_security_policy(
            name=network_policy_name,
            policy=json.dumps([{
                'Rules': [{'Resource': ['collection/' + vector_store_name], 'ResourceType': 'collection'}],
                'AllowFromPublic': True
            }]),
            type='network'
        )
    except Exception as ex:
        print(ex)
    
    try:
        access_policy = aoss_client.create_access_policy(
            name=access_policy_name,
            policy=json.dumps([{
                'Rules': [
                    {
                        'Resource': ['collection/' + vector_store_name],
                        'Permission': ['aoss:CreateCollectionItems', 'aoss:DeleteCollectionItems', 'aoss:UpdateCollectionItems', 'aoss:DescribeCollectionItems'],
                        'ResourceType': 'collection'
                    },
                    {
                        'Resource': ['index/' + vector_store_name + '/*'],
                        'Permission': ['aoss:CreateIndex', 'aoss:DeleteIndex', 'aoss:UpdateIndex', 'aoss:DescribeIndex', 'aoss:ReadDocument', 'aoss:WriteDocument'],
                        'ResourceType': 'index'
                    }
                ],
                'Principal': [role_arn],
                'Description': 'Easy data policy'
            }]),
            type='data'
        )
    except Exception as ex:
        print(ex)
        
    return encryption_policy, network_policy, access_policy

In [ ]:
# Create Collection
vector_store_name = f'nova-mm-image-collection-{suffix}'
collection_name = vector_store_name

encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=vector_store_name,
    aoss_client=aoss_client,
    role_arn=role_arn
)

In [ ]:
# Create collection (if not exists)
try:
    response = aoss_client.create_collection(
        name=collection_name,
        type='VECTORSEARCH',
        description='Product multimodal search collection'
    )
    print(f"Collection '{collection_name}' created successfully.")
    
    # Wait for collection to be active and get endpoint
    while True:
        collections = aoss_client.list_collections(collectionFilters={'name': collection_name})
        if collections['collectionSummaries'][0]['status'] == 'ACTIVE':
            collection_id = collections['collectionSummaries'][0]['id']
            collection_details = aoss_client.batch_get_collection(ids=[collection_id])
            collection_endpoint = collection_details['collectionDetails'][0]['collectionEndpoint']
            break
        time.sleep(10)
        
    print(f"Collection endpoint: {collection_endpoint}")
    
except Exception as e:
    if 'ConflictException' in str(e):
        print(f"Collection '{collection_name}' already exists.")
        collections = aoss_client.list_collections(collectionFilters={'name': collection_name})
        collection_id = collections['collectionSummaries'][0]['id']
        collection_details = aoss_client.batch_get_collection(ids=[collection_id])
        collection_endpoint = collection_details['collectionDetails'][0]['collectionEndpoint']
        print(f"Collection endpoint: {collection_endpoint}")
    else:
        print(f'Failed to create collection: {e}')
        raise

# Wait a bit more for the endpoint to be fully ready
print("Waiting for collection endpoint to be ready...")
time.sleep(30)

In [ ]:
# Extract host from collection endpoint
host = collection_endpoint.replace('https://', '')

# Create OpenSearch client
client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# Define index mapping for vector search
index_body = {
    "settings": {
        "index": {
            "knn": True,
            "knn.algo_param.ef_search": 100
        }
    },
    "mappings": {
        "properties": {
            "combined_vector": {
                "type": "knn_vector",
                "dimension": dim,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "nmslib"
                }
            },
            "item_id": {"type": "keyword"},
            "item_name": {"type": "text"},
            "img_path": {"type": "keyword"},
            "type": {"type": "keyword"}
        }
    }
}

# Create index
try:
    response = client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created successfully.")
except Exception as e:
    if 'resource_already_exists_exception' in str(e):
        print(f"Index '{index_name}' already exists.")
    else:
        print(f'Failed to create index: {e}')
        raise

## 2. Load Amazon Berkeley Objects Dataset

In [ ]:
# Load product metadata
meta = pd.read_json("s3://amazon-berkeley-objects/listings/metadata/listings_0.json.gz", lines=True)

def func_(x):
    us_texts = [item["value"] for item in x if item["language_tag"] == "en_US"]
    return us_texts[0] if us_texts else None

meta = meta.assign(item_name_in_en_us=meta.item_name.apply(func_))
meta = meta[~meta.item_name_in_en_us.isna()][["item_id", "item_name_in_en_us", "main_image_id"]]
print(f"#products with US English title: {len(meta)}")
meta.head()

In [ ]:
# Load image metadata and merge
image_meta = pd.read_csv("s3://amazon-berkeley-objects/images/metadata/images.csv.gz")
dataset = meta.merge(image_meta, left_on="main_image_id", right_on="image_id")

# Create full S3 path for images
dataset = dataset.assign(img_full_path='s3://amazon-berkeley-objects/images/small/' + dataset.path.astype(str))
print(f"Final dataset size: {len(dataset)}")
dataset.head()

## 3. Generate Embeddings for Products

In [ ]:
def generate_text_embedding(text, purpose="GENERIC_INDEX"):
    """Generate embedding for text"""
    request_body = {
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingDimension": dim,
            "embeddingPurpose": purpose,
            "text": {"truncationMode": "END", "value": text}
        }
    }
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    result = json.loads(response['body'].read())
    return result["embeddings"][0]["embedding"]

import base64

def load_file_as_base64(s3_uri):
    """Download file from S3 and convert to base64"""
    bucket = s3_uri.split('/')[2]
    key = '/'.join(s3_uri.split('/')[3:])
    
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    return base64.b64encode(obj['Body'].read()).decode('utf-8')

def generate_image_embedding(s3_uri, purpose="GENERIC_INDEX"):
    """Generate embedding for image"""
    request_body = {
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingPurpose": purpose,
            "embeddingDimension": dim,
            "image": {
                "format": "jpeg",
                "detailLevel": "STANDARD_IMAGE",
                "source": {"bytes": load_file_as_base64(s3_uri)},
            },
        },
    }
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body)
    )
    
    result = json.loads(response['body'].read())
    return result["embeddings"][0]["embedding"]

In [ ]:
# Process products and generate embeddings
import numpy as np
from tqdm import tqdm

# Take first 100 products for demo
sample_dataset = dataset.head(100)
batch_size = 10
documents = []

for idx, row in tqdm(sample_dataset.iterrows(), total=len(sample_dataset), desc="Processing products"):
    try:
        # Generate text embedding
        text_embed = generate_text_embedding(row['item_name_in_en_us'])
        
        # Generate image embedding
        img_embed = generate_image_embedding(row['img_full_path'])
        
        # Combine embeddings (average)
        combined_embed = [(t + i) / 2 for t, i in zip(text_embed, img_embed)]
        
        # Create document for OpenSearch
        doc = {
            "combined_vector": combined_embed,
            "item_id": row['item_id'],
            "item_name": row['item_name_in_en_us'],
            "img_path": row['img_full_path'],
            "type": "combined"
        }
        
        documents.append(doc)
        
        # Batch index to OpenSearch
        if len(documents) >= batch_size:
            for doc in documents:
                client.index(
                    index=index_name,
                    body=doc
                )
            print(f"Indexed batch ending with item {row['item_id']}")
            documents = []
            
    except Exception as e:
        print(f"Error processing item {row['item_id']}: {e}")
        continue

# Index remaining documents
if documents:
    for doc in documents:
        client.index(
            index=index_name,
            body=doc
        )
    print("Indexed final batch")

## 4. Test Search Functionality

In [ ]:
def search_products(query_embed, topK=3):
    """Search products using embedding"""
    search_body = {
        "size": topK,
        "query": {
            "knn": {
                "combined_vector": {
                    "vector": query_embed,
                    "k": topK
                }
            }
        }
    }
    
    response = client.search(
        index=index_name,
        body=search_body
    )
    
    return response

def display_results(search_response):
    """Display search results with images"""
    from IPython.display import display, HTML
    
    html_content = ""
    for i, hit in enumerate(search_response['hits']['hits']):
        source = hit['_source']
        score = hit['_score']
        
        # Generate presigned URL for image
        bucket = source['img_path'].split('/')[2]
        key = '/'.join(source['img_path'].split('/')[3:])
        img_url = boto3.client('s3').generate_presigned_url(
            'get_object', Params={'Bucket': bucket, 'Key': key}, ExpiresIn=3600
        )
        
        html_content += f"""
        <div style="border: 1px solid #ccc; margin: 10px; padding: 10px; display: flex; align-items: center;">
            <img src="{img_url}" style="width: 150px; height: 150px; object-fit: contain; margin-right: 20px;">
            <div>
                <h3>{i+1}. {source['item_name']}</h3>
                <p><strong>Type:</strong> {source['type']} | <strong>Score:</strong> {score:.3f}</p>
                <p><strong>Item ID:</strong> {source['item_id']}</p>
            </div>
        </div>
        """
    
    display(HTML(html_content))

### a. Query with Image

In [ ]:
# Test image query - use an image from the dataset
test_image_path = sample_dataset.iloc[0]['img_full_path']

# Display query image
from IPython.display import display, HTML
bucket = test_image_path.split('/')[2]
key = '/'.join(test_image_path.split('/')[3:])
query_img_url = boto3.client('s3').generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': key}, ExpiresIn=3600)
display(HTML(f'<h3>Query Image:</h3><img src="{query_img_url}" style="width: 200px; height: 200px; object-fit: contain; border: 2px solid #007acc;">'))

query_embed = generate_image_embedding(test_image_path)
results = search_products(query_embed)
print("\nImage Search Results:")
display_results(results)

### b. Query with Text Only

In [ ]:
# Test text query
query_text = "15-ounce drinkware"
print(f"Searching for: {query_text}")

query_embed = generate_text_embedding(query_text, "IMAGE_RETRIEVAL")
results = search_products(query_embed)
print("\nText Search Results:")
display_results(results)

### c. Query with Hybrid Image and Text

In [ ]:
# Test hybrid query
query_text = "AmazonBasics drinkware set"
test_image_path = dataset.iloc[101]['img_full_path']  # Different image

# Display query image
bucket = test_image_path.split('/')[2]
key = '/'.join(test_image_path.split('/')[3:])
query_img_url = boto3.client('s3').generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': key}, ExpiresIn=3600)
display(HTML(f'<h3>Hybrid Query - Text: "{query_text}"</h3><img src="{query_img_url}" style="width: 200px; height: 200px; object-fit: contain; border: 2px solid #007acc;">'))

text_embed = generate_text_embedding(query_text, "GENERIC_RETRIEVAL")
img_embed = generate_image_embedding(test_image_path, "GENERIC_RETRIEVAL")

# Combine embeddings (average)
hybrid_embed = [(t + i) / 2 for t, i in zip(text_embed, img_embed)]

results = search_products(hybrid_embed)
print("\nHybrid Search Results:")
display_results(results)

## 5. Cleanup Resources

In [ ]:
# delete vector index
client.indices.delete(index=index_name)

# delete data, network, and encryption access ploicies
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

# delete collection
aoss_client.delete_collection(id=collection_id)